# Simple Sports/eSports League Scheduling Model

This notebook contains a model for creating a schedule for a simple sports league. I used this problem to create a schedule for a Pokemon Draft League for a group of YouTubers.

# Problem Definition

Consider a sports style league with the following characteristics:
* There are 16 teams
* There are 10 weeks in the season
* Each team plays 1 match each week
* Each match has a home team and an away team
* Each team has 5 home games and 5 away games throughout the season
* Two teams can only face-off against each other at most once in a season

# Model Formulation

## Sets

$ w \in W$: The set of weeks (time periods) \
$ t \in T$: The set of teams

## Decision Variables

$ x_{wha}$: A binary variable that equals 1 if team $h \in T$ is home against team $a \in T, a \neq h$, during week $ w \in W$ and 0 otherwise

## Model

### Objective Function

$Min \space \space 0 \qquad$  (there is no objective, we just want a feasible schedule)

### Constraints

Each team plays exactly one match each week:

$$ \sum_{a \in T, a \neq i} x_{wia} + \sum_{h \in T, h \neq i} x_{whi} = 1, \qquad \forall w \in W, i \in T$$ 

Each team is the home team 5 times throughout the season:

$$ \sum_{w \in W}\sum_{a \in T, a \neq i} x_{wia} = 5, \qquad \forall i \in T$$

Each team is the away team 5 times throughout the season:

$$ \sum_{w \in W}\sum_{h \in T, h \neq i} x_{whi} = 5, \qquad \forall i \in T$$

Two teams can only face off against each other at most once during the season:

$$ \sum_{w \in W} x_{wij} + \sum_{w \in W} x_{wji} \leq 1, \qquad \forall i \in T, j \in T, j \neq i$$ 

# Import PuLP

PuLP is a great Python Library for formulating mixed integer programs

In [1]:
import pulp

# Sets

$ t \in T$: The set of teams

In [2]:
T = ["AshtonCox",
     "ck49",
     "Fevzi",
     "JinFurai",
     "JoeUX9",
     "Mandby",
     "MoxieBoosted",
     "Nekkra",
     "NinoPokeBros",
     "OsirusVGC",
     "PKMNcast",
     "PokeAlex",
     "SierraDawn",
     "StephOfAnime",
     "TheBattleRoom",
     "Viz"]

$ w \in W$: The set of weeks (time periods)

In [3]:
W = range(1,11)

# Decision Variables

$ x_{wha}$: A binary variable that equals 1 if team $h \in T$ is home against team $a \in T, a \neq h$, during week $ w \in W$ and 0 otherwise

In [4]:
x = pulp.LpVariable.dicts('x',(W,T,T),cat=pulp.LpBinary)

The previous cell created extra decision variables that we don't want. These are $ x_{wha}$ variables in which $h=a$. Because a team cannot play against itself, we need to remove them.

In [5]:
for w in W:
    
    for h in T:
        
        for a in T:
            
            if h == a:
                
                del x[w][h][a]

# Create the Problem in PuLP

In [6]:
schedule = pulp.LpProblem("Schedule",pulp.LpMinimize)

# Create the Objective Function

We are only concerned about creating a schedule that satisfies all of the constraints. We are not optimizing anything. Thus, we can simply set the objective function to 0.

In [7]:
schedule += 0

# Constraints

Each team plays exactly one match each week:

$$ \sum_{a \in T, a \neq i} x_{wia} + \sum_{h \in T, h \neq i} x_{whi} = 1, \qquad \forall w \in W, i \in T$$ 

In [8]:
for w in W:
    
    for i in T:
        
        schedule += pulp.lpSum(x[w][i][a] for a in T if a != i) + pulp.lpSum(x[w][h][i] for h in T if h != i) == 1

Each team is the home team 5 times throughout the season:

$$ \sum_{w \in W}\sum_{a \in T, a \neq i} x_{wia} = 5, \qquad \forall i \in T$$

In [9]:
for i in T:
    
    schedule += pulp.lpSum(x[w][i][a] for w in W for a in T if a != i) == 5

Each team is the away team 5 times throughout the season:

$$ \sum_{w \in W}\sum_{h \in T, h \neq i} x_{whi} = 5, \qquad \forall i \in T$$

In [10]:
for i in T:
    
    schedule += pulp.lpSum(x[w][h][i] for w in W for h in T if h != i) == 5

Two teams can only face off against each other at most once during the season:

$$ \sum_{w \in W} x_{wij} + \sum_{w \in W} x_{wji} \leq 1, \qquad \forall i \in T, j \in T, j \neq i$$ 

In [11]:
for i in T:
    
    for j in T:
        
        if j != i:
            
            schedule += pulp.lpSum(x[w][i][j] for w in W) + pulp.lpSum(x[w][j][i] for w in W) <= 1

# Solve the Problem

In [12]:
schedule.solve()

1

# Print the results

In [13]:
for w in W:
    
    for h in T:
        
        for a in T:
            
            if a != h:
                
                if x[w][h][a].value() == 1.0:
                    
                    
                    week, home, away = x[w][h][a].getName().split('_')[1:]
                    
                    print(f'Week: {week}  {home} vs. {away}')
                    
    print()

Week: 1  AshtonCox vs. Viz
Week: 1  JinFurai vs. Nekkra
Week: 1  JoeUX9 vs. StephOfAnime
Week: 1  Mandby vs. Fevzi
Week: 1  MoxieBoosted vs. PKMNcast
Week: 1  PokeAlex vs. OsirusVGC
Week: 1  SierraDawn vs. NinoPokeBros
Week: 1  TheBattleRoom vs. ck49

Week: 2  ck49 vs. PKMNcast
Week: 2  Fevzi vs. StephOfAnime
Week: 2  Mandby vs. NinoPokeBros
Week: 2  MoxieBoosted vs. AshtonCox
Week: 2  Nekkra vs. TheBattleRoom
Week: 2  OsirusVGC vs. JinFurai
Week: 2  SierraDawn vs. JoeUX9
Week: 2  Viz vs. PokeAlex

Week: 3  AshtonCox vs. Fevzi
Week: 3  ck49 vs. Viz
Week: 3  JinFurai vs. NinoPokeBros
Week: 3  JoeUX9 vs. OsirusVGC
Week: 3  Mandby vs. MoxieBoosted
Week: 3  PKMNcast vs. SierraDawn
Week: 3  PokeAlex vs. Nekkra
Week: 3  StephOfAnime vs. TheBattleRoom

Week: 4  AshtonCox vs. ck49
Week: 4  Fevzi vs. Viz
Week: 4  Nekkra vs. Mandby
Week: 4  NinoPokeBros vs. OsirusVGC
Week: 4  PKMNcast vs. JoeUX9
Week: 4  PokeAlex vs. MoxieBoosted
Week: 4  StephOfAnime vs. JinFurai
Week: 4  TheBattleRoom vs. Sier